Problema:

Propor um classificador que identifique o grupo de cada objeto.

Dados:

* $g$: número de grupos diferentes
* $n$: número de objetos (não necessariamente diferentes)
* $n_{min}$: número mínimo de objetos em um grupo
* $n_{max}$: número máximo de objetos em um grupo

Para cada Objeto:

* $c$: número de características binárias
* $c_y$: número de características de um determinado grupo
* $c_n$: número de características dos demais grupos ($c_n = c_y (g - 1)$)
* $p$: probabilidade de ativação das características de um grupo ($p > 0.5$)
* $1 - p$: probabilidade de ativação das características dos demais grupos
* $p' = 0.5$: probabilidade de ativação das características que não são de qualquer grupo
* (as características de cada grupo não tem interseção)

## Gerador de Instância

In [1]:
"gera a distribuição de objetos para os grupos"
function group_size(g, n, n_min, n_max)
    num_g = Array(Int, g)
    sum = 0
    for i=1:g
        num_g[i] = rand(n_min:n_max)
        sum += num_g[i]
    end
    correct = n / sum
    sum = 0
    for i=1:g
        num_g[i] = round(Int, num_g[i] * correct)
        sum += num_g[i]
    end
    if sum < n
        num_g[g] += 1
    end
    num_g
end

"máscara de características para cada grupo sem interseção"
function group_mask(g, c, c_y)
    char_g = fill(-1, c)
    index = 1
    for i=1:g, j=1:c_y
        char_g[index] = i
        index += 1
    end
    char_g
end

"""gera objetos para grupos seguindo a distribuição num_g,
a máscara char_g e a probabilidade p de ativação"""
function generate_data(num_g, char_g, p)
    data = Array(Tuple{Array{Int,1},Int}, 0)
    for i=1:length(num_g),j=1:num_g[i]
        vect = zeros(Int, length(char_g))
        for k=1:length(vect)
            if char_g[k] == i
                vect[k] = rand() < p ? 1 : 0
            elseif char_g[k] != -1
                vect[k] = rand() < 1 - p ? 1 : 0
            else
                vect[k] = rand() < 0.5 ? 1 : 0
            end
        end
        push!(data, (vect, i))
    end
    data
end

"gerador de instâncias para o problema de clusterização"
function oc152_t_instance_generator(n, c, c_y, p, g, n_min, n_max)
    num_g = group_size(g, n, n_min, n_max)
    char_g = group_mask(g, c, c_y)
    data = generate_data(num_g, char_g, p)
    data
end

oc152_t_instance_generator (generic function with 1 method)

In [2]:
function data(;n = 20, n_min = 0, n_max = 0, g = 5, c = 16, c_y = 3, p = 0.8)
    if n < 10
        println("minimum 10")
        return nothing
    end
    if g > n
        println("too many groups")
        return nothing
    end
    if c < g * c_y
        println("c_y too big")
        return nothing
    end
    
    if n_max == 0
        n_max = 2 * round(Int, n / g)
    end
    if n_min == 0
        n_min = round(Int, n_max / 10)
    end
    if n_max * g < n
        println("n_max too tight")
        return nothing
    end
    
    data = oc152_t_instance_generator(n, c, c_y, p, g, n_min, n_max)
    shuffle!(data)
end

data (generic function with 1 method)

### K-Means

Consiste em executar o algoritmo *K-means* determinar os pontos *centrais* de cada grupo e classificar cada objeto como sendo do grupo com ponto central *mais próximo*

#### Algoritmo Iterativo

1. Choose $k$ cluster centers randomly generated in a domain containing all the points,
2. Assign each point to the closest cluster center,
3. Recompute the cluster centers using the current cluster memberships,
4. If a convergence criterion is met, stop; Otherwise go to step 2.

In [30]:
n = 20
k = 5
train_data = data(n = n, g = k)

inputs = map(v -> float(v[1]), train_data)

# inicialização com amostragem sem reposição de k objetos como centros iniciais
means = map(i -> inputs[i], randperm(length(inputs))[1:k])

"função que calcula o índice do centro de menor distância de v"
classify(v) = indmin(map(c -> norm(c - v), means))

assignments = nothing
iters = 0

while true
    iters += 1
    # calcula o centro associado a cada objeto
    new_assignments = map(classify, inputs)
    
    # encerra o processamento se não tiver mudança com a última iteração
    assignments == new_assignments && break
    
    # recalcula os centros como a média dos pontos do último agrupamento
    assignments = new_assignments

    #println("Centros ", iters, ": ", means)
    #println("Agrupamentos ", iters, ": ", new_assignments)
    
    for i=1:k
        # lista todos os objetos do i-ésimo agrupamento
        i_points = map(ii -> inputs[ii], findin(assignments, i))
        
        isempty(i_points) && continue
        means[i] = mean(i_points)
    end
end

println("Número de grupos: ", k)
println("Número de objetos: ", length(inputs))
println("Número de iterações: ", iters)

groups = map(v -> v[2], train_data)
for i=1:k
    g_index = findin(groups, i)
    centers = map(i -> assignments[i], g_index)
    counts = hist(centers, 0:k)[2]
    println("Group ", i, ": (", indmax(counts), ") ", counts)
end
sleep(0.2)

Número de grupos: 5
Número de objetos: 20
Número de iterações: 2
Group 1: (3) [0,1,2,2,0]
Group 2: (5) [0,2,0,0,4]
Group 3: (1) [2,0,0,0,0]
Group 4: (4) [0,0,0,3,0]
Group 5: (2) [0,3,1,0,0]


#### Mixed Integer Programming with Nonlinear Objective

$$
\begin{align}
\text{minimize} \qquad & \sum_{j=1}^{k}\sum_{i=1}^{n} x_{ij} || s_i - c_j||^2 \\
 \text{subject to} \quad \quad & \sum_{j=1}^{k} x_{ij} = 1 & i = 1, \ldots, n \\
 \qquad \qquad & \sum_{i=1}^{n} x_{ij} \ge 1 & j = 1, \ldots, k \\
 \qquad \qquad & x_{ij} \in \{0, 1\} & i = 1, \ldots, n; j = 1, \ldots, k \\
\end{align}
$$

$$
\begin{align}
c_j = \frac{\sum_{l=1}^{n} x_{lj} s_l}{\sum_{l=1}^{n} x_{lj}}
\end{align}
$$

#### Approximate Semidefinite programming (SDP) relaxation

$$
\begin{align}
\text{minimize} \qquad & Tr(W(I-Z)) \\
 \text{subject to} \quad \quad & Ze = e \\
 \qquad \qquad & Tr(Z) = k \\
 \qquad \qquad & Z \succeq 0\\
\end{align}
$$

$$
\begin{align}
W_{ij} = \phi(s_i,s_j) = exp^{-\frac{||s_i-s_j||^2}{\sigma}}, \sigma > 0
\end{align}
$$

In [47]:
#sum(Z_val, 1)

1x20 Array{Float64,2}:
 1.0  0.999995  0.999999  0.999997  …  0.999999  1.0  1.0  0.999996  1.0

In [55]:
using JuMP

p = length(inputs)

W = zeros(p,p)
for i=1:p, j=i+1:p
    dist = exp(-norm(inputs[i] - inputs[j])/1.0)
    W[i,j] = dist
    W[j,i] = dist
end

m = Model()

@defVar(m, Z[1:p,1:p], SDP)
@addConstraint(m, Z .≥ 0)

@setObjective(m, Min, trace(W * (eye(p,p) - Z)))

@addConstraint(m, Z * ones(p) .== ones(p))

@addConstraint(m, trace(Z) == k)

solve(m)

Z_val = getValue(Z)[:,:]

println("Raw solution")
println(round(Z_val,4))

# A simple rounding scheme
which_cluster = zeros(Int,p)
num_clusters = 0
for i = 1:p
    Z_val[i,i] <= 1e-6 && continue

    if which_cluster[i] == 0
        num_clusters += 1
        which_cluster[i] = num_clusters
        for j = i+1:p
            println(num_clusters, " ", round(norm(Z_val[i,j] - Z_val[i,i]),2))
            if norm(Z_val[i,j] - Z_val[i,i]) <= 1e-1
                which_cluster[j] = num_clusters
            end
        end
    end
end

println("Clusters: ", which_cluster)
# Print results
for cluster = 1:k
    println("Cluster $cluster")
    for i = 1:p
        if which_cluster[i] == cluster
            println(i, "->", inputs[i])
        end
    end
end

sleep(0.2)

Raw solution
[0.2332 0.2332 -0.0 0.0 0.2332 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 0.0 0.0 -0.0 -0.0 0.0671 0.2332 0.0 -0.0
 0.2332 0.2332 0.0 0.0 0.2332 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0671 0.2332 0.0 0.0
 -0.0 0.0 0.2274 0.0 0.0 0.067 0.0 0.0 0.0 0.045 0.1695 0.2246 0.0 0.0 0.0245 0.1272 0.0108 0.0 0.0 0.1039
 0.0 0.0 0.0 0.4657 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0687 0.0 0.4657 0.0
 0.2332 0.2332 0.0 0.0 0.2332 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0671 0.2332 0.0 0.0
 -0.0 0.0 0.067 0.0 0.0 0.0931 0.0802 0.1112 0.0835 0.0791 0.0787 0.0676 0.0 0.0 0.1078 0.0872 0.0528 -0.0 0.0 0.0919
 -0.0 0.0 0.0 0.0 0.0 0.0802 0.3665 0.0204 0.3808 0.0038 -0.0 0.0 0.0174 0.0174 -0.0 0.0 0.1134 -0.0 0.0 -0.0
 -0.0 0.0 0.0 0.0 0.0 0.1112 0.0204 0.2052 0.0213 0.126 0.0549 0.0027 0.0 0.0 0.1925 0.095 0.0536 -0.0 0.0 0.1172
 -0.0 0.0 0.0 0.0 0.0 0.0835 0.3808 0.0213 0.3964 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1181 -0.0 0.0 0.0
 -0.0 0.0 0.045 0.0 0.0 0.0791 0.0038 0.126 -0.0 0